In [ ]:
#Imports---------------------------------------------------------------
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image
#%matplotlib inline
#%config InlineBackend.figure_format = 'retina'

import glob
import numpy as np
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
#Main pipeline for multiclass classification
#import libraries
import numpy as np
from time import time
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset
from tqdm import tqdm
from sklearn.model_selection import KFold
from torchvision import models, transforms
import pickle
import os
import sys
from PIL import Image


In [ ]:
#Parameters---------------------------------------------------------------
import torch.nn.functional as F

K=10
arc='wide_resnet50_2'
gamma=1 #Loss Parameters
thresh=0.95 #Threshold
mu=7 #Parameter of unlabeled vs labeled --batch size
lr=0.03
b_s=64
ep=50
wd=0.0005
init=10 #Number of epochs that start training the model


In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#proportion = [0.546,0.2602,0.08129,0.108,0.0038]
proportion = torch.tensor([0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1])
proportion = proportion.to(device)
#mov_avg=mov_avg(K,100)



In [ ]:
#Define Transforms
cifar10_mean = (0.4914, 0.4822, 0.4465)
cifar10_std = (0.2471, 0.2435, 0.2616)

transform = transforms.Compose(
    [ 
      transforms.RandomHorizontalFlip(),
      transforms.RandomVerticalFlip(),
      transforms.ToTensor(),
      transforms.Normalize(mean=cifar10_mean, std=cifar10_std)])

transform_strong = transforms.Compose([
    transforms.RandomHorizontalFlip(),
      transforms.RandomVerticalFlip(),              
      transforms.RandAugment(8,6),
      transforms.ToTensor(),                     
      transforms.Normalize(mean=cifar10_mean, std=cifar10_std)])


In [ ]:
##Dataset CIFAR 10 Feito por mim-------------------------------------------------------------------------------
##-------------------------------------------------------------------------------------------------------------
import random
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,download=True,)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,transform=transform,download=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=b_s,shuffle=False, num_workers=2)
print(len(trainset))
counter=[0]*10
counter_u=[0]*10
#random.shuffle(trainset)

labset=[]
labset_l=[]
unlabset=[]
labset_x=[]
labset_y=[]
for i in range (0,49999):
  for z in range(0,10):
    if(trainset[i][1]==z):
      if(counter[z]<=24):
        counter[z]=counter[z]+1
        labset_x.append(trainset[i][0])
        labset_y.append(trainset[i][1])

      else:
      #if(counter[z]>24 and counter_u[z]<=1999):
                counter_u[z]=counter_u[z]+1
                print(trainset[i][0])
                unlabset.append(trainset[i][0])  
print(counter)
print(len(labset))
print(len(unlabset)) 


In [ ]:
class MyDataset(Dataset):
   def __init__(self, type, transform,strong):
       if(type=='lab'):
        self.X,self.Y=labset_x,labset_y
        self.transform = transform
        self.type=type
       else:
        self.X = unlabset
        self.transform = transform
        self.strong = strong
        self.type=type

   def __getitem__(self, i):
       if(self.type == 'lab'):
        X=self.transform(self.X[i])
        Y=self.Y[i]
       else:
         X=self.transform(self.X[i])
         Y=self.strong(self.X[i])
       return X,Y

   def __len__(self):
       return len(self.X)

In [ ]:
ds= MyDataset('lab',transform,transform)
lab = DataLoader(ds, b_s, True,num_workers=1)
#v_ds= MyDataset('val',transform,'/content/drive/My Drive/ML/CVT_1/Transform',1)
#val = DataLoader(v_ds, b_s, True)
t_ds= MyDataset('unl',transform,transform_strong)
unl = DataLoader(t_ds, b_s*7, True,num_workers=1)

In [ ]:
# Model
class Base(nn.Module):
    def __init__(self, pretrained_model, n_outputs):
        super().__init__()
        self.n_outputs = n_outputs
        model = getattr(models, pretrained_model)(pretrained=True) #using transfer learning from a pretrained model - True
        model = nn.Sequential(*tuple(model.children())[:-1])
        last_dimension = torch.flatten(model(torch.randn(1, 3, 224, 224))).shape[0]
        self.model = nn.Sequential(
            model,
            nn.Flatten(),
            nn.Dropout(0.2),
            nn.Linear(last_dimension, 1024),
            nn.Dropout(0.2),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.Dropout(0.2),
            nn.ReLU(),
            nn.Linear(512,256),
            nn.ReLU(),
            nn.Linear(256, K),

        )

    def forward(self, x):
        return self.model(x)

    def loss(self, Yhat, Y):
        return loss_func(Yhat, Y)

    def to_proba(self, Yhat):
        return F.softmax(Yhat, 1)

    def to_classes(self, Phat):
        return Phat.argmax(1)


In [ ]:
#Give to the model the architecture and number of classes (K)
model = Base(arc, K)
#put the model on 'GPU' or 'CPU'
model = model.to(device)
#Define optimizer and scheduler 
#Define optimizer and scheduler 
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9,weight_decay=wd)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True)


In [ ]:
import gc
#Define Moving Average
class mov_avg:
    def  __init__(self,num_classes,num_points):
        self.num_classes=num_classes
        self.num_points=num_points
        queue= []
        for i in range(500):
            queue.append(0)
            queue.append(1)
            queue.append(2)
            queue.append(3)
            queue.append(4)
            queue.append(5)
            queue.append(6)
            queue.append(7)
            queue.append(8)
            queue.append(9)
        self.queue=queue
        self.res=torch.tensor([100,100,100,100,100,100,100,100,100,100])
        print(len(queue))
    def add(self,point):     
            a=self.queue.pop(0)

            self.queue.append(point)

            self.res[a]=self.res[a]-1
            self.res[point]=self.res[point]+1
            
    def get_value(self,type):
        x=0
        for i in range(self.num_points):
          if(self.queue[i]==type):
                x=x+1
         
        x = x/self.num_points      
    def get_values(self):
        t=torch.zeros(10)
        for i in range (10):
          t[i]=self.res[i]/self.num_points
        return t
proportion=proportion.to(device)
mov_avg=mov_avg(K,1000)











#Initialize variables
running_loss=0
kl_loss = nn.KLDivLoss(reduction="mean")
l_test=len(testloader)*64

for epoch in range(ep):
    #Print Statistics
    print('\n')
    print('Iteração:',epoch)
    print('loss:',running_loss)
    avg_acc=0
    total_correct=0.0

    if(epoch%1==0):
        model.eval()

        print('Test')
        test_end=0
        test_iter=iter(testloader)  
        try:  
            inputs,labels = test_iter.next()
        except StopIteration:
            end=1  

        while(test_end==0):
        #for inputs,labels in test 
            inputs,labels = inputs.to(device),labels.to(device)
            outputs = model(inputs)  
            #Lx = F.cross_entropy(outputs, labels, reduction='mean')    
            _, predicted = torch.max(outputs, 1)
            real=labels
            correct = (predicted == real).float().sum()
            avg_acc +=correct*100/l_test


            total_correct=total_correct + correct

            print('.',end='')
            running_loss=0.0
            try:  
                inputs,labels = test_iter.next()
            except StopIteration:
                test_end=1
        model.train()        
    print('')     
    print('Totais',total_correct) 
    print('Acc',avg_acc) 
    end=0
    running_loss=0
    unlabloader = iter(unl)
    labloader = iter(lab)
    while(end==0):
        #Get batches---------
        try:
            weak,strong = unlabloader.next()
        except StopIteration:
            end=1
        if(end==0):
            try:
                inputs, labels = labloader.next()
            except StopIteration:
                labloader = iter(lab)
                inputs, labels = labloader.next()
            inputs,labels=inputs.to(device),labels.to(device)
            weak,strong=weak.to(device),strong.to(device)

            #Zero the optimizer
            optimizer.zero_grad()

            #Get outputs of the model
            l_l=len(inputs)
            l_w=len(weak)
            l_s=len(strong)
            test=torch.cat((inputs,weak,strong),dim=0)
            del inputs,weak,strong
            test=test.to(device)
            outputs=model(test)    
                
            outputs2 = outputs[:l_l]
            weak_o = outputs[l_l:l_l+l_w]
            strong_o = outputs[l_l+l_w:l_l+2*l_s]

            for x in outputs:
                #print(x)
                _,ind=torch.max(x,0)  
                mov_avg.add(ind)
            
            #Calculate Labeled Losses
            Lx = F.cross_entropy(outputs2,labels,reduction='mean')
            Lu=0
            #Go into unlabeled batches
            for i in range(len(weak_o)):
              #calculate alpha and y
              ma=mov_avg.get_values()
              ma=ma.to(device)
              w=weak_o[i]
              s=strong_o[i]
              q=torch.softmax(w,0)
              q=q*proportion/ma
              max,y=torch.max(q,0)
              alpha=1-max
              mov_avg.add(y)
              #check if s is inside q
              #If not calculate p^
              s_hat=torch.softmax(s,0)
              S=[]
              P=[]
              Lu=0
              count=0
              if(s_hat[y]<1-alpha):
                p=torch.zeros(10)
                count+=1
                for z in range(10):
                  if(z==y):
                    p[z]=1-alpha
                  else:
                    p[z]=alpha*s_hat[z]/(1-s_hat[y])
                s=torch.log_softmax(s,0)
                p=torch.softmax(p,0)
                s=s.to(device)
                p=p.to(device)
                S.append(s)
                P.append(p)      
              
            if(count>0):
              S=torch.stack(S,1)
              P=torch.stack(P,1)
              Lu=kl_loss(S,P)    
            #Calculate loss and optimize the CNN  
            if(Lu>0):  
              loss=Lx+Lu
              #print('Lu')
            else:
              loss=Lx
            print('.',end='')
            loss.backward()
            optimizer.step()    
            running_loss+=loss.item()   
    scheduler.step(running_loss)


5000
10048


Iteração: 0
loss: 0
Test
.............................................................................................................................................................
Totais tensor(2881., device='cuda:0')
Acc tensor(28.6724, device='cuda:0')
................................................................................................................

Iteração: 1
loss: 15.133571755606681
Test
.............................................................................................................................................................
Totais tensor(2812., device='cuda:0')
Acc tensor(27.9857, device='cuda:0')
................................................................................................................

Iteração: 2
loss: 22.966392451897264
Test
.............................................................................................................................................................
Totais tensor(2936., devic